In [1]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)
print(f"Project root added to sys.path: {project_root}")

Project root added to sys.path: c:\Users\sylla\Desktop\MLOPS\Project1_with_deployment


In [2]:
%pwd

'c:\\Users\\sylla\\Desktop\\MLOPS\\Project1_with_deployment\\Project1_Data_Science\\research'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\sylla\\Desktop\\MLOPS\\Project1_with_deployment\\Project1_Data_Science'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir : Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    alpha : float
    random_state : int
    l1_ratio : float
    target_column : str

In [5]:
from src.Project1_Data_Science.constants import *
from src.Project1_Data_Science.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        # Ensure the artifacts root directory exists
       
        create_directories([config.root_dir])


        # Create the model trainer config
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha=params.alpha,
            random_state=params.random_state,
            l1_ratio=params.l1_ratio,
            target_column= schema.name
        )
        
        return model_trainer_config
        


In [7]:
import pandas as pd
import os
from src.Project1_Data_Science import logger
from sklearn.linear_model import ElasticNet
import joblib

In [8]:
class ModelTrainer :
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        logger.info(f"Training model with config: {self.config}")
        
        # Load training data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        X_train = train_data.drop(columns=[self.config.target_column])
        X_test = test_data.drop(columns=[self.config.target_column])

        y_train = train_data[[self.config.target_column]]

        y_test = test_data[[self.config.target_column]]

        # Train the model
        model_en = ElasticNet(
            alpha=self.config.alpha,
            random_state=self.config.random_state,
            l1_ratio=self.config.l1_ratio
        )
        
        model_en.fit(X_train, y_train)

        # Save the model
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(model_en, model_path)
        
        logger.info(f"Model saved at: {model_path}")

In [9]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e


Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\config\config.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\params.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\schema.yaml
2025-04-09 10:55:13,817 - Datasciencelogger - INFO - created directory at: artifacts
2025-04-09 10:55:13,824 - Datasciencelogger - INFO - created directory at: artifacts/model_trainer
2025-04-09 10:55:13,828 - Datasciencelogger - INFO - Training model with config: ModelTrainerConfig(root_dir='artifacts/model_trainer', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_name='model.joblib', alpha=0.2, random_state=42, l1_ratio=0.1, target_column='quality')
2025-04-09 10:55:13,980 - Datasciencelogger - INFO - Model saved at: artifacts/model_trainer\model.joblib
